# Actividad 4 | Métricas de calidad de resultados
Identificar métricas para la medición de la calidad de resultados derivados de la aplicación de modelos de aprendizaje, ya sea supervisado o no supervisado, orientado al procesamiento de grandes volúmenes de datos, que permitan la selección de los modelos que mejor se ajusten a la tarea de aprendizaje a resolver.



## Alejandro González Almazán - A00517113


--------------------------------------------------------------------------------

# Impotacion de Librerias

In [1]:
# PySpark
import findspark
findspark.init()
findspark.find()
"""
# omitir para ejecutar de forma local
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
"""
#Librerias de codigo
import kagglehub


### Inicializar entorno PySpark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Analisis_Steam") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark

### Lectura de datos

In [3]:
file_path = kagglehub.dataset_download("najzeko/steam-reviews-2021")

In [4]:
print(file_path)

C:\Users\alexa\.cache\kagglehub\datasets\najzeko\steam-reviews-2021\versions\1


In [5]:
print(f"{file_path}/steam_reviews.csv")

C:\Users\alexa\.cache\kagglehub\datasets\najzeko\steam-reviews-2021\versions\1/steam_reviews.csv


In [6]:
df = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("sep", ",").option("escape", "\"").csv(f"{file_path}/steam_reviews.csv")

new_columns = [col_name.replace(".", "_") for col_name in df.columns]
df = df.toDF(*new_columns)

## 1. Construcción de la muestra M (Particiones Mi) con PySpark
Para construir una muestra M representativa de la población P (dataset de reseñas de Steam), se generarán particiones Mi basadas en variables de caracterización clave. El objetivo es asegurar que cada Mi mantenga proporcionalidad respecto a la distribución original, evitando sesgos.

#### Variables de Particionamiento
Se utilizaron las siguientes variables categóricas identificadas en el análisis previo:

1. **recommended:** Si el usuario recomienda el juego (True/False).

2. **received_for_free:** Si el juego fue recibido gratis (True/False).

3. **language:** Idioma de la reseña (ej. english, spanish, russian).

#### Cálculo de Fracciones de Muestreo por Estrato

Para construir una muestra `M` representativa y de tamaño controlado (`TAMAÑO_M`):

1. **Fracción por estrato**:
   - Se calcula como `(TAMAÑO_M * proporción_original) / conteo_original`.
   - Esto asegura que cada estrato contribuya a `M` en proporción a su peso en el dataset original.

2. **Límite del 100%**:
   - Si un estrato es demasiado pequeño para contribuir al tamaño deseado sin superar el 100%, se incluye completo.

3. **Diccionario de estratos**:
   - Clave: Tupla `(language, received_for_free, recommended)`.
   - Valor: Fracción de muestreo (ej: `0.3` para tomar el 30% del estrato).

In [7]:
from pyspark.sql.functions import col

# Calcular conteos y proporciones originales
original_counts = df.groupBy("language", "received_for_free", "recommended") \
    .count() \
    .withColumn("proportion", col("count") / df.count())

# Mostrar estratos y sus proporciones
original_counts.orderBy("proportion", ascending=False).show(5)

+--------+-----------------+-----------+-------+--------------------+
|language|received_for_free|recommended|  count|          proportion|
+--------+-----------------+-----------+-------+--------------------+
| english|            false|       true|8335990| 0.38331024012051845|
|schinese|            false|       true|2832942|  0.1302659526064093|
| russian|            false|       true|1998749| 0.09190761494803211|
| english|            false|      false|1020063| 0.04690511786459154|
|schinese|            false|      false| 844851|0.038848419884867924|
+--------+-----------------+-----------+-------+--------------------+
only showing top 5 rows



In [8]:
from pyspark.sql.functions import col, when, concat_ws

# --------------------------------------------------------
# Paso 1: Definir tamaño objetivo de la muestra M
# --------------------------------------------------------
TARGET_SAMPLE_SIZE = 2_000_000  # Puedes ajustar este valor según tu caso

# --------------------------------------------------------
# Paso 2: Calcular la fracción de muestreo para cada estrato
# (estratificando por: language, received_for_free, recommended)
# --------------------------------------------------------

strata_df = original_counts.withColumn(
    "fraction",
    (TARGET_SAMPLE_SIZE * col("proportion")) / col("count")
).withColumn(
    "fraction",
    col("fraction").cast("double")
).withColumn(
    "fraction",
    when(col("fraction") > 1.0, 1.0).otherwise(col("fraction"))
)

strata_df.show(5)

+---------+-----------------+-----------+-------+--------------------+-------------------+
| language|received_for_free|recommended|  count|          proportion|           fraction|
+---------+-----------------+-----------+-------+--------------------+-------------------+
|  turkish|            false|       true| 552688|0.025414014411213198|0.09196513914256578|
|  turkish|            false|      false|  59662| 0.00274341206576188| 0.0919651391425658|
|  english|            false|      false|1020063| 0.04690511786459154| 0.0919651391425658|
|bulgarian|            false|       true|   8723|4.011059543703007E-4| 0.0919651391425658|
|    czech|            false|       true| 119043|0.005473903029474229|0.09196513914256578|
+---------+-----------------+-----------+-------+--------------------+-------------------+
only showing top 5 rows



In [9]:
# --------------------------------------------------------
# Paso 3: Crear columna clave combinada para sampleBy()
# --------------------------------------------------------
# Esto nos permite usar sampleBy() incluso si la estratificación
# se basa en múltiples columnas.
strata_df = strata_df.withColumn(
    "stratum_key",
    concat_ws("_", "language", "received_for_free", "recommended")
)

# Crear también la columna clave en el DataFrame completo
df = df.withColumn(
    "stratum_key",
    concat_ws("_", "language", "received_for_free", "recommended")
)

+---+------+--------------------+---------+--------+----------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+-------------------+
|_c0|app_id|            app_name|review_id|language|                            review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|        stratum_key|
+---+------+--------------------+---------+--------+----------------------------------+-----------------+-----------------+---

In [10]:
# --------------------------------------------------------
# Paso 4: Convertir las fracciones de muestreo a diccionario
# --------------------------------------------------------
strata_fractions = {
    row["stratum_key"]: row["fraction"]
    for row in strata_df.select("stratum_key", "fraction").collect()
}

# --------------------------------------------------------
# Paso 5: Aplicar sampleBy() con la columna stratum_key
# --------------------------------------------------------
# Esto genera automáticamente la muestra estratificada M
# sin necesidad de usar múltiples filtros ni uniones.

sample_M = df.sampleBy(
    "stratum_key",
    fractions=strata_fractions,
    seed=42
)

# --------------------------------------------------------
# Paso 6 (opcional): Verificar tamaño y distribución de la muestra M
# --------------------------------------------------------
print(f"Tamaño total de la muestra M: {sample_M.count()}")
sample_M.groupBy("language", "received_for_free", "recommended").count().show(5)

Tamaño total de la muestra M: 2001342
+----------+-----------------+-----------+-----+
|  language|received_for_free|recommended|count|
+----------+-----------------+-----------+-----+
|   turkish|            false|       true|51115|
|   english|            false|      false|93911|
|     czech|            false|       true|10928|
|   russian|             true|       true| 8791|
|   turkish|            false|      false| 5435|
| brazilian|            false|      false| 4288|
|    polish|            false|      false| 2328|
|   english|             true|       true|22760|
|   spanish|             true|       true| 2395|
|   finnish|             true|       true|  150|
| bulgarian|            false|       true|  788|
| bulgarian|            false|      false|   79|
|   swedish|             true|       true|  275|
| norwegian|             true|       true|  127|
|  japanese|             true|       true|  178|
|   spanish|             true|      false|  187|
|  romanian|             true| 

## 2. Construcción del conjunto de entrenamiento y prueba
### 2.1 Limpieza de datos

Antes de constuir el conjunto de entrenamiento y prueba es necesario realizar una limpieza de los datos

In [11]:
from pyspark.sql.functions import col, when, length, isnan, count

# --------------------------------------------------------
# Paso 1: Eliminar filas con valores nulos en columnas clave
# --------------------------------------------------------
clean_sample_M = sample_M.dropna(subset=["recommended", "author_playtime_forever", "votes_helpful", "review"])

# --------------------------------------------------------
# Paso 2: Truncar valores outliers de 'votes_helpful'
# --------------------------------------------------------
clean_sample_M = clean_sample_M.withColumn(
    "votes_helpful",
    when(col("votes_helpful") > 1000, 1000).otherwise(col("votes_helpful"))
)

# --------------------------------------------------------
# Paso 3: Convertir variables booleanas a numéricas
# --------------------------------------------------------
clean_sample_M = clean_sample_M.withColumn(
    "recommended_numeric",
    when(col("recommended") == True, 1).otherwise(0)
).drop("recommended")

# --------------------------------------------------------
# ✅ Paso 4: Crear nueva columna de longitud de la reseña
# --------------------------------------------------------
clean_sample_M = clean_sample_M.withColumn("review_length", length(col("review")))

# --------------------------------------------------------
# Paso 5 (opcional): Validaciones rápidas
# --------------------------------------------------------
print("Validación de datos después del preprocesamiento:")
clean_sample_M.select("recommended_numeric", "votes_helpful", "review_length").show(5)

# Validar nulos en columnas críticas
clean_sample_M.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c)
    for c in ["author_playtime_forever", "votes_helpful", "review_length"]
]).show()


Validación de datos después del preprocesamiento:
+-------------------+-------------+-------------+
|recommended_numeric|votes_helpful|review_length|
+-------------------+-------------+-------------+
|                  1|            0|            2|
|                  1|            0|           18|
|                  1|            0|           59|
|                  1|            0|           30|
|                  1|            0|           19|
+-------------------+-------------+-------------+
only showing top 5 rows

+-----------------------+-------------+-------------+
|author_playtime_forever|votes_helpful|review_length|
+-----------------------+-------------+-------------+
|                      0|            0|            0|
+-----------------------+-------------+-------------+



### 2.2 Preparación de la muestra - vectorización de features
Antes de entrenar un modelo de Machine Learning con PySpark, debemos convertir las variables predictoras en un vector numérico. Este paso es crucial porque los algoritmos de ML de Spark requieren que todas las variables de entrada estén en una sola columna de tipo Vector.

#### ¿Qué es VectorAssembler?
Es una herramienta de PySpark que permite:
   - Tomar múltiples columnas individuales (números o variables codificadas)
   - Agruparlas en una única columna llamada features (tipo Vector)

#### Selección de variables
**Variable objetivo (target):** Es la que queremos predecir con nuestro modelo.
En este caso, es `recommended_numeric`, que indica si un usuario recomendó el producto (1) o no (0).

**Variables predictoras (features):** Son las columnas que el modelo usará para hacer esa predicción.
   - Cuántas horas jugó el usuario (`author_playtime_forever`)
   - Cuántos votos de utilidad recibió la reseña (`votes_helpful`)
   - Qué tan larga fue la reseña (`review_length`)


In [12]:
from pyspark.ml.feature import VectorAssembler

# Variables predictoras
predictor_columns = [
    "author_playtime_forever",
    "votes_helpful",
    "review_length",
    "received_for_free"
]

# Creamos el VectorAssembler
assembler = VectorAssembler(
    inputCols=predictor_columns,
    outputCol="features"
)

# Aplicamos el transformador al DataFrame limpio
vectorized_df = assembler.transform(clean_sample_M).cache() # Cache for faster development
vectorized_df.count()  # Triggers caching immediately

1998216

In [13]:
vectorized_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- app_name: string (nullable = true)
 |-- review_id: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: integer (nullable = true)
 |-- timestamp_updated: long (nullable = true)
 |-- votes_helpful: long (nullable = true)
 |-- votes_funny: long (nullable = true)
 |-- weighted_vote_score: double (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- steam_purchase: boolean (nullable = true)
 |-- received_for_free: boolean (nullable = true)
 |-- written_during_early_access: boolean (nullable = true)
 |-- author_steamid: long (nullable = true)
 |-- author_num_games_owned: long (nullable = true)
 |-- author_num_reviews: long (nullable = true)
 |-- author_playtime_forever: double (nullable = true)
 |-- author_playtime_last_two_weeks: double (nullable = true)
 |-- author_playtime_at_review: double (nullable = true)
 |-- au

### 2.3 Construcción Train – Test
**Objetivo:** Dividir el conjunto M (la muestra estratificada) en dos particiones:
   * Conjunto de entrenamiento: Tri
   * Conjunto de prueba: Tsi

Asegurando:
   * Que cada estrato Mi conserve su proporción.
   * Que no haya intersección entre Tri y Tsi.
   * Que su unión forme M: Tri ∪ Tsi = M

Nos apoyaremos en la variable `stratum_key` generada previamente para estratificación, la cual representa las combinaciones de variables de caracterización.


In [14]:
# Check for nulls or blanks
vectorized_df.filter("stratum_key IS NULL").count()
vectorized_df.filter("stratum_key = ''").count()

# Sample only non-null values
vectorized_df.filter("stratum_key IS NOT NULL").select("stratum_key").distinct().show()

+--------------------+
|         stratum_key|
+--------------------+
|   french_false_true|
| tchinese_false_true|
|  koreana_false_true|
|brazilian_false_f...|
|  swedish_false_true|
|  polish_false_false|
|vietnamese_false_...|
|   koreana_true_true|
|     latam_true_true|
|   danish_false_true|
|tchinese_false_false|
|    german_true_true|
|  english_true_false|
|japanese_false_false|
| italian_false_false|
|bulgarian_false_f...|
|   swedish_true_true|
| norwegian_true_true|
|   latam_false_false|
| swedish_false_false|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import col

# Semilla para reproducibilidad
SEED = 42
train_fraction = 0.8  # 80% entrenamiento, 20% prueba

# 1️⃣ Estratificación: obtenemos todos los valores únicos del estrato
stratum_values = vectorized_df.select("stratum_key").distinct().collect()

# 2️⃣ Inicializamos DataFrames vacíos para acumulación
train_parts = []
test_parts = []

# 3️⃣ Para cada estrato Mi, dividimos usando randomSplit para evitar sesgos
for stratum in stratum_values:
    stratum_df = vectorized_df.filter(col("stratum_key") == stratum)
    train_df, test_df = stratum_df.randomSplit([train_fraction, 1 - train_fraction], seed=SEED)
    train_parts.append(train_df)
    test_parts.append(test_df)

# 4️⃣ Unimos todos los conjuntos de entrenamiento y prueba
train_set = train_parts[0]
for df in train_parts[1:]:
    train_set = train_set.union(df)

test_set = test_parts[0]
for df in test_parts[1:]:
    test_set = test_set.union(df)

# 5️⃣ Verificación de integridad
print("Entrenamiento:", train_set.count())
print("Prueba:", test_set.count())
print("Total:", vectorized_df.count())
print("¿Intersección vacía?", train_set.intersect(test_set).count() == 0)